In [2]:
from E2E_vanilla import Vanilla_E2E

In [5]:
model = Vanilla_E2E()

Using cache found in /home/jbinda/.cache/torch/hub/facebookresearch_dino_main


In [33]:
vars(model.dino.blocks[0].attn)

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('qkv',
               Linear(in_features=384, out_features=1152, bias=True)),
              ('attn_drop', Dropout(p=0.0, inplace=False)),
              ('proj', Linear(in_features=384, out_features=384, bias=True)),
              ('proj_drop', Dropout(p=0.0, inplace=False))]),
 'num_heads': 6,
 'scale': 0.125}

In [16]:
print(model)

Vanilla_E2E(
  (dino): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(8, 8), stride=(8, 8))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (norm): LayerNorm((3

In [1]:
import math
from functools import partial

import torch.nn.functional as F
import torch
import torch.nn as nn
from timm.models.layers import DropPath, to_2tuple, to_3tuple

In [2]:
%load_ext autoreload
%autoreload 2

from dino3d import VisionTransformer3D

In [3]:
force_2d = False
img_size = (5, 512, 512)
patch_size = 16
bootstrap_method = "centering"
encoder = VisionTransformer3D(
    img_size=img_size if not force_2d else (1, img_size[1], img_size[2]),
    patch_size=(patch_size, patch_size, img_size[0])
    if not force_2d
    else (patch_size, patch_size, 1),
    embed_dim=768,
    depth=12,
    in_chans=1,
    num_heads=12,
    mlp_ratio=4,
    qkv_bias=True,
    norm_layer=partial(nn.LayerNorm, eps=1e-6),
)
encoder.init_weights(bootstrap_method=bootstrap_method)
x = torch.randn(4, 1, 5, 512, 512)
out = encoder(x)
print([y.shape for y in out])

Old: torch.Size([384, 3, 8, 8]) tensor(-1.8925)
Centering!!!
New: torch.Size([384, 1, 8, 8, 5]) tensor(-1.8925)


RuntimeError: Error(s) in loading state_dict for VisionTransformer3D:
	size mismatch for cls_token: copying a param with shape torch.Size([1, 1, 384]) from checkpoint, the shape in current model is torch.Size([1, 1, 768]).
	size mismatch for pos_embed: copying a param with shape torch.Size([1, 1025, 384]) from checkpoint, the shape in current model is torch.Size([1, 1025, 768]).
	size mismatch for patch_embed.proj.weight: copying a param with shape torch.Size([384, 1, 8, 8, 5]) from checkpoint, the shape in current model is torch.Size([768, 1, 16, 16, 5]).
	size mismatch for patch_embed.proj.bias: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for blocks.0.norm1.weight: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for blocks.0.norm1.bias: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is

In [41]:
print(encoder)

VisionTransformer3D(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(1, 768, kernel_size=(16, 16, 5), stride=(16, 16, 5))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (head): Identit

## Docelowe

In [13]:
img_size = (7, 224, 224)
patch_size = 8
bootstrap_method = "centering"
encoder = VisionTransformer3D(
    img_size=img_size,
    patch_size=(patch_size, patch_size, img_size[0]),
    embed_dim=384,
    depth=12,
    in_chans=1,
    num_heads=6,
    mlp_ratio=4,
    qkv_bias=True,
    norm_layer=partial(nn.LayerNorm, eps=1e-6),
)

In [14]:
encoder.init_weights(bootstrap_method=bootstrap_method)
x = torch.randn(4, 1, 7, 224, 224)
out = encoder(x)
print([y.shape for y in out])

Old: torch.Size([384, 3, 8, 8]) tensor(-1.8925)
Centering!!!
New: torch.Size([384, 1, 8, 8, 7]) tensor(-1.8925)
Pretrained weights found at ./pretrained_models/dino_deitsmall8_pretrain.pth and loaded with msg: <All keys matched successfully>
[torch.Size([384]), torch.Size([384]), torch.Size([384]), torch.Size([384])]


In [16]:
print(encoder)

VisionTransformer3D(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(1, 384, kernel_size=(8, 8, 7), stride=(8, 8, 7))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()


### Closer look on the model

In [7]:
from torchinfo import summary

In [17]:
batch_size = 32
summary(encoder, input_size=(batch_size, 1, 7, 224, 224),
        col_names=["input_size",
                "output_size",
                "num_params",
                "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VisionTransformer3D                      [32, 1, 7, 224, 224]      [32, 384]                 301,824                   True
├─PatchEmbed: 1-1                        [32, 1, 7, 224, 224]      [32, 784, 384]            --                        True
│    └─Conv3d: 2-1                       [32, 1, 224, 224, 7]      [32, 384, 28, 28, 1]      172,416                   True
├─Dropout: 1-2                           [32, 785, 384]            [32, 785, 384]            --                        --
├─ModuleList: 1-3                        --                        --                        --                        True
│    └─Block: 2-2                        [32, 785, 384]            [32, 785, 384]            --                        True
│    │    └─LayerNorm: 3-1               [32, 785, 384]            [32, 785, 384]            768                       True
│    

In [50]:
from E2E_3DViT_dataset import E2E_3D_Dataset

In [51]:
train_ds = E2E_3D_Dataset(mode="train")

In [60]:
ds = torch.utils.data.DataLoader(train_ds, batch_size=32)

In [61]:
loader = iter(ds)

In [63]:
b =next(loader)

In [64]:
b[0].shape

torch.Size([32, 1, 7, 32, 32])